## Import Python module

In [1]:
import matplotlib.pyplot as plt
import time

In [2]:
import numpy as np 
import cv2
from pywt import dwt,idwt
import os

#### 임시
***

In [3]:
phone = "galaxyS7"
# phone = "iPhoneX"
p = "phone1"
img_num = 1

***

### 1. LSB

In [4]:
def watermark_lsb(x):
    
    x = format(x, '#010b')
    temp = x[:-2] + '11'
    result = int(temp, 2)
    
    return np.uint8(result)

In [5]:
url = 'pic/{}/img{}.png'.format(phone,img_num)
# url = 'pic/lena.png'

lsb_img= cv2.imread(url)
img_b = lsb_img[:,:,0].copy()

col_mid = img_b.shape[1]//2
row_mid = img_b.shape[0]//2

# 두께 4, 가로세로 삽입
for i in range(0,img_b.shape[0]):
    for k in range(col_mid-2,col_mid+2):
        img_b[i,k] = watermark_lsb(img_b[i,k])
        
for i in range(row_mid-2,row_mid+2):
    for k in range(0,img_b.shape[1]):
        img_b[i,k] = watermark_lsb(img_b[i,k])
        
lsb_img[:,:,0] = img_b.copy()

### 2. Embed

In [6]:
def block_add_wm(block,i):
    
    mod = 32
    
    i = i%(wm_shape[0]*wm_shape[1])

    wm_1 = wm_flatten[i]
    
    block_dct = cv2.dct(block)
    
    U,s,V = np.linalg.svd(block_dct)
    
    max_s = s[0]

    s[0] = (max_s-max_s%mod+7/8*mod) if wm_1>=128 else (max_s-max_s%mod+1/8*mod)

    block_dct_embed = np.dot(U,np.dot(np.diag(s),V))

    return cv2.idct(block_dct_embed)

In [7]:
start_emb = time.time()

ori_img = lsb_img.astype(np.float32)
ori_img_shape = ori_img.shape[:2]

ori_img_YUV = cv2.cvtColor(ori_img, cv2.COLOR_BGR2YUV)

dwt_deep = 1 

if not ori_img_YUV.shape[0]%(2**dwt_deep)==0:
    temp = (2**dwt_deep)-ori_img_YUV.shape[0]%(2**dwt_deep)
    ori_img_YUV = np.concatenate((ori_img_YUV,np.zeros((temp,ori_img_YUV.shape[1],3))),axis=0)
    
if not ori_img_YUV.shape[1]%(2**dwt_deep)==0:
    temp = (2**dwt_deep)-ori_img_YUV.shape[1]%(2**dwt_deep)
    ori_img_YUV = np.concatenate((ori_img_YUV,np.zeros((ori_img_YUV.shape[0],temp,3))),axis=1)
    
# assert ori_img_YUV.shape[0]%(2**dwt_deep)==0
# assert ori_img_YUV.shape[1]%(2**dwt_deep)==0

# dwt1
coeffs_Y = dwt(ori_img_YUV[:,:,0],'haar')
ha_Y = coeffs_Y[0]

coeffs_Y = coeffs_Y[1]

# block
block_shape=(8,8)
ha_block_shape = (int(ha_Y.shape[0]/block_shape[0]),int(ha_Y.shape[1]/block_shape[1]),block_shape[0],block_shape[1])
strides = ha_Y.itemsize*(np.array([ha_Y.shape[1]*block_shape[0],block_shape[1],ha_Y.shape[1],1]))

ha_Y_block = np.lib.stride_tricks.as_strided(ha_Y.copy(),ha_block_shape,strides)
# coeffs_Y_block = np.lib.stride_tricks.as_strided(coeffs_Y.copy(),ha_block_shape,strides)

# wm
wm_url = "wm/{}_img{}.png".format(p,img_num)
wm = cv2.imread(wm_url)[:,:,0]
wm_shape = wm.shape[:2]

# 이미지 크기가 충분히 큰지 확인
img_shape = ha_Y.shape
shape0_int,shape1_int = img_shape[0]//block_shape[0], img_shape[1]//block_shape[1]

if not shape0_int*shape1_int>=wm_shape[0]*wm_shape[1]:
    print("워터마크의 크기가 사진의 용량을 초과")

part_shape = (shape0_int*block_shape[0],shape1_int*(block_shape[1]))
block_index0,block_index1 = np.meshgrid(np.arange(shape0_int),np.arange(shape1_int))
b_flat_idx0, b_flat_idx1 = block_index0.flatten(),block_index1.flatten()

# 길이
length = b_flat_idx0.size

#워터마크 flatten
wm_flatten = wm.flatten()

# embed
embed_Y_block = ha_Y_block.copy()
# cY_block = coeffs_Y_block.copy()

# start_emb = time.time()

for i in range(length):
    embed_Y_block[b_flat_idx0[i], b_flat_idx1[i]] = block_add_wm(embed_Y_block[b_flat_idx0[i], b_flat_idx1[i]], i)
#     cY_block[b_flat_idx0[i], b_flat_idx1[i]] = block_add_wm(cY_block[b_flat_idx0[i], b_flat_idx1[i]], i)
    
# print("Time:",time.time()-start_emb)

# re-shape
embed_Y_part = np.concatenate(embed_Y_block,1)
embed_Y = np.concatenate(embed_Y_part,1)

# embed_cY_part = np.concatenate(cY_block,1)
# cY = np.concatenate(embed_cY_part,1)

ha_Y[:part_shape[0],:part_shape[1]] = embed_Y
# coeffs_Y[:part_shape[0],:part_shape[1]] = cY

# idwt
embed_ha_Y_idwt = idwt(ha_Y.copy(), coeffs_Y,'db1',"smooth")



ori_img_YUV[:,:,0] = embed_ha_Y_idwt
embed_img_YUV = ori_img_YUV[:ori_img_shape[0],:ori_img_shape[1]]

img_float32 = np.float32(embed_img_YUV)
embed_img = cv2.cvtColor(img_float32,cv2.COLOR_YUV2BGR)

embed_img[embed_img>255]=255
embed_img[embed_img<0]=0


# cv2.imwrite("embed/captured_lena.png",embed_img)

cv2.imwrite("embed/captured_{}_img{}.png".format(p,img_num),embed_img)

print("Time:",time.time()-start_emb)

Time: 1.9557380676269531
